# Air Sac Tracking Prototype code
Lara S. Burchardt & Wim Pouw

lara.burchardt@donders.ru.nl

Next to body movements and acoustics, we would also like to track the air sac's inflation of the Siamangs. The air sac naturally forms a spherical(3D) or circular (2D) shape, and such shapes are retrievable from an image using the hough transform, and a bit of pre-processing of the images to get the optimal representation of the relevent edges of the air sac.

This code takes as input a sample video with a close up of a Siamang, and then tracks the air sac when it takes a sufficiently circular shape. The result is shown below; it is not perfect, but with a bit of smoothing this can function as a good air sac tracker. This code is very much under development, there are many ways to improve further.

In [1]:
# import the necessary packages
import numpy as np
import argparse
import cv2
import pandas as pd
from skimage import io, feature, color, measure, draw, img_as_float
import numpy as np
import csv
import random2
import statistics
import scipy
from scipy import signal
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import itertools
import os
from os import listdir
from os.path import isfile, join


#resused code from 
#https://pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
#https://stackoverflow.com/questions/31705355/how-to-detect-circlular-region-in-images-and-centre-it-with-python

## Define Prepocessing Function

In [2]:
#define parameters for HoughTransform outside of function to be able to save and manipulate easier
def hougdraw(submitted_image, dp = 1, mindist = 10000, param1 = 10, param2=22, minradius = 5, maxradius=250):
    circles = cv2.HoughCircles(submitted_image, cv2.HOUGH_GRADIENT, 
                               dp = dp,minDist = mindist,  
                               param1 = param1, param2 = param2, 
                               minRadius = minradius, maxRadius = maxradius)
    if circles is not None:
        circles = np.round(circles[0, 0:1]).astype("int")
        circle1 = circles[0,0]
        circle2 = circles[0,1]
        circle3 = circles[0,2]
        for(x, y, r) in circles:
            cv2.circle(submitted_image, (x, y), r, (255, 255, 0), 2) #version without drawing roi back on whole image 
    return(submitted_image)
    
# define function for preprocessing
def preprocessing(image, phase1_medianblur = 27, 
                              phase2_dilate = 7, 
                              phase2_medianblur = 19, 
                              alpha = 2, 
                              beta= 30,
                              param1canny=10,
                              param2canny=15):
    #image0 = hougdraw(image)
    #convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    #brightness change
    gray = cv2.convertScaleAbs(gray, alpha = alpha, beta = beta)
    #set dynamic tresholds for canny (and thus also for hough)
    mean_intensity = np.median(gray)
    threshold1 = int(max(0, (1.0 - 0.33) * mean_intensity/param1canny))
    threshold2 = int(min(255, (1.0 + 0.33) * mean_intensity/param2canny))    
    image1_h = hougdraw(gray,param1= threshold1, param2 =threshold2)
    #blur
    image2 = cv2.medianBlur(gray, phase1_medianblur)
    image2_h = hougdraw(image2.copy(),param1= threshold1, param2 =threshold2)
    #dynamic thresholds for canny edge detection based on intensity of image
    #Thresholds one standard deviation above and below median intensity
    #edge detection
    image3 = cv2.Canny(image2, threshold1, threshold2)
    image3_h = hougdraw(image3.copy(), param1= threshold1, param2 =threshold2)
    #dilation and second blur
    submitted = cv2.dilate(image3, None, iterations= phase2_dilate)  
    image4 = cv2.medianBlur(submitted, phase2_medianblur) 
    image4_h = hougdraw(image4.copy(), param1= threshold1, param2 =threshold2)
    #add hough
    return image1_h, image2_h, image3_h, image4_h
############################################

def preprocessing2(image, phase1_medianblur = 27, 
                              phase2_dilate = 5, 
                              phase2_medianblur = 19, 
                              alpha = 2, 
                              beta= 30,
                              param1canny=10,
                              param2canny=15):
    #image0 = hougdraw(image)
    #convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    #brightness change
    gray = cv2.convertScaleAbs(gray, alpha = alpha, beta = beta)
    #set dynamic tresholds for canny (and thus also for hough)
    mean_intensity = np.median(gray)
    threshold1 = int(max(0, (1.0 - 0.33) * mean_intensity/param1canny))
    threshold2 = int(min(255, (1.0 + 0.33) * mean_intensity/param2canny))    
    #blur
    image2 = cv2.medianBlur(gray, phase1_medianblur)
    #dynamic thresholds for canny edge detection based on intensity of image
    #Thresholds one standard deviation above and below median intensity
    #edge detection
    image3 = cv2.Canny(image2, param1canny, param2canny)
    #dilation and second blur
    submitted = cv2.dilate(image3, None, iterations= phase2_dilate)  
    image4 = cv2.medianBlur(submitted, phase2_medianblur) 
    #add hough
    image4 = np.float32(image4)
    return image4, threshold1, threshold2

def matriximages(outputname, wimagenumber, himagenumber, imagelist, marginbetweenimages, vertnames, hornames):
    """
    matrix images takes in the full path name of the outputfolder for storing the images
    it takes as input a list of full path names of the images
    the height is given in image numbers and same for width
    the vert names is a list of column names
    the hor names is a list of row names
    """
    margin = marginbetweenimages
    #img = imagelist  
    w = wimagenumber
    h = himagenumber
    n = w*h

    #imgs = [cv2.imread(i) for i in imagelist]
    imgs = imagelist
    #if any(i.shape != imgs[0].shape for i in imgs[1:]):
    #    raise ValueError('Not all images have the same shape.')

    img_h, img_w = imgs[0].shape#img_h, img_w, img_c = imgs[0].shape

    m_x = 0
    m_y = 0
    if marginbetweenimages is not None:
        margin = marginbetweenimages
        m_x = int(margin*img_w)
        m_y = int(margin*img_h)
        
    imgmatrix = np.zeros((img_h * h + m_y * (h - 1),
                          img_w * w + m_x * (w - 1)),
                         np.uint8)

    imgmatrix.fill(255)    

    positions = itertools.product(range(w), range(h))
    for (x_i, y_i), img in zip(positions, imgs):
        x = x_i * (img_w + m_x)
        y = y_i * (img_h + m_y)
        imgmatrix[y:y+img_h, x:x+img_w] = img #imgmatrix[y:y+img_h, x:x+img_w, :] = img
    #add text
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(imgmatrix, hornames[0], (0*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[1], (1*img_w,  int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[2], (2*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[3], (3*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[4], (4*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[0], (0, 1*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[1], (0, 2*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[2], (0, 3*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[3], (0, 4*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    #cv2.putText(imgmatrix, vertnames[4], (0, 5*img_h), font, 10, (0, 255, 0), 5, cv2.LINE_AA)
    cv2.imwrite(outputname, imgmatrix)   
    
    print('done, look in your folder: '+ outputname)

## Main Function

This function is also used in the looper: process_all_videos.py

The tracker is run on the full video, it first defines an ROI defined in the first step. 
The output is saved in a table format and as a video with the detected circles drawn on top. 

In [12]:
# preprocessing, choosing a roi on the tracking results of 50 random samples from input file
#https://github.com/patchy631/machine-learning/blob/main/computer_vision/cv2_edge_detection.ipynb

############settings that worked: c1_5_c2_10_al_1_b_12_dil_7_blur_27
alpha = 1
beta = 12
dp = 1
dilation = 7
phase1_medianblur = 27
cannyt1 = 5
cannyt2 = 12 
minDist = 10000
minRadius = 5 #minus 2times std
maxRadius = 300 #dynamic? 
###################
videofolder= '../Video/'
nameforfiles = 'example8'
videofilename = nameforfiles +'.mp4'
# Opens the Video file
cap = cv2.VideoCapture(videofolder+videofilename)
totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)   #fps = frames per second

#set up empty output dataframe
column_names = ['x','y', 'r', 'frame'] # we are going to collect hough generated x,y,r,frame from random set of frames
df_round_1 = pd.DataFrame(columns = column_names) 
vector = range(0, int(totalFrames), 1)
samp = random2.sample(vector, 50) #50 is the setting

for rand in samp:
    # set frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES,rand)
    ret, frame = cap.read()
    ############################detect circles   
    output=frame.copy()
    # transform to grayscale image only using the roi part of the image
    image4, param1, param2 = preprocessing2(image=output)
    final_im = cv2.normalize(src=image4, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    circles = cv2.HoughCircles(final_im, cv2.HOUGH_GRADIENT, 
                               dp = dp,minDist = minDist,  
                               param1 = param1,param2 = param2, 
                               minRadius = minRadius, maxRadius = maxRadius)
    #if circles is not None:
    #    circles = np.round(circles[0, 0:1]).astype("int")
    if circles is not None:
        #circles = np.round(circles[0, 0:1]).astype("int")
        if circles is not None:
            circles = np.round(circles[0, 0:1]).astype("int")
            circle1 = circles[0,0]
            circle2 = circles[0,1]
            circle3 = circles[0,2]
        #save it to a row
        if circles is None:
            circle1 = "NA"
            circle2 = "NA"
            circle3 = "NA"
    new_row = [circles[0,0], circles[0,1],circles [0,2], rand]
    df_round_1.loc[len(df_round_1)] = new_row
    cv2.waitKey(1)
cv2.destroyAllWindows()

#when collected take some info
median_x = df_round_1['x'].median() #order in pos : same as in original dataframe, so x,y, r
median_y = df_round_1['y'].median()
max_r = df_round_1['r'].max()
min_r = df_round_1['r'].min()
    # order of parameters saved as roi with cv2.selectROI [Top_Left_X, Top_Left_Y, Width, Height]
   
    # explanation: height_1 is the maximum radius detected plus 30 pixels, that is used to determine the position of the 
    # upper left corner of the roi, same goes for width_1, both are currently the same, but that could change, so they are both
    # coded independently
height_1 = max_r + 100
width_1 = max_r + 250
pos_x = median_x - width_1
pos_y = median_y - height_1  
# to be consistent with the roi nomenclature, we then calculate the width/height of the whole roi which is width_1 *2
    # all those values will then be used to crop picture in next round of tracking
width_2 = width_1 * 2
height_2 = height_1 *2
#
roi = [pos_x, pos_y, width_2, height_2]

# cleaning up
#cap.set(cv2.CAP_PROP_POS_FRAMES,samp[1])
#ret, frame = cap.read()
#cv2.rectangle(frame,(int(roi[0]), int(roi[1])), (int(roi[0]+roi[2]), int(roi[1]+roi[3])), (0,0,0), 2) 
#cv2.imshow("output", frame) #ROI)
cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()

################################

# Opens the Video file
cap = cv2.VideoCapture(videofolder+videofilename)
frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)   #fps = frames per second

#output video
out = cv2.VideoWriter('./Output/videos/output_'+videofilename,cv2.VideoWriter_fourcc(*'MP4V'), fps, 
                      (int(frameWidth), int(frameHeight)))
#set up empty output dataframe
column_names = ['frame','roi_x1','roi_y1','roi_x2', 'roi_y2', # info on region of interest for repetability
                'x','y', 'r', 'inputvideo','input_id',                    # actual results 
                'alpha', 'beta',                              # values of brightness and contrast pre processing
                'dp', 'minDist', 'param1', 'param2', 'minRadius',' maxRadius'] # parameters of hough circle transform 
df = pd.DataFrame(columns = column_names)
print("parameters set")
#initialize iterator
i=0
#ROI as automatically detected beforehand
pos_x =0 
pos_y= 0
width_2=0
height_2=0

#main loop                      
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
  ############################detect circles   
    output=frame.copy()
    print("preprocessing image")
    image4, param1, param2 = preprocessing2(image=output)
    print("done")
    #track demicircles 
    name = 'framenr_' + str(i+1) + '_framevid_' + videofilename[0:-4] 
    # original values: dp = 1, minDist = 10000, param1=1, param2=10, minRadius = 5, maxRadius= 250
    #submit to 
    image4 = image4[int(roi[1]):int(roi[1]+roi[3]), int(roi[0]):int(roi[0]+roi[2])]
    final_im = cv2.normalize(src=image4, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) 
    circles = cv2.HoughCircles(final_im, cv2.HOUGH_GRADIENT, 
                               dp = dp,minDist = minDist,  
                               param1 = param1,param2 = param2, minRadius = minRadius, maxRadius = maxRadius)   
    #cv2.rectangle(image, start_point, end_point, color, thickness)    
    cv2.rectangle(frame,(int(roi[0]), int(roi[1])), (int(roi[0]+roi[2]), int(roi[1]+roi[3])), (0,0,0), 2)    

    if circles is not None:
        #circles = np.round(circles[0, 0:1]).astype("int")
        if circles is not None:
            circles = np.round(circles[0, 0:1]).astype("int")
            circle1 = circles[0,0]+roi[0] #x  + plus the shift from the roi
            circle2 = circles[0,1]+roi[1] #y  + plus the shift from the roi
            circle3 = circles[0,2]
    #    #save it to a row
        if circles is None:
            circle1 = "NA"
            circle2 = "NA"
            circle3 = "NA"
        # code for drawing the found circles onto the original video    
        for (x, y, r) in circles:
            #cv2.circle(frame, (x, y), r, (255, 255, 0), 2) #version without drawing roi back on whole image
            #cv2.circle(output[int(roi[1]): int(roi[1]+roi[3]),
            #         int(roi[0]):int(roi[0]+roi[2])], (x, y), r, (255, 255, 0), 2)
            cv2.circle(frame, (int(x+roi[0]), int(y+roi[1])), r, (200, 0, 0), 2)
            
            #print(x,y,r)
    cv2.waitKey(1)
    out.write(frame)
    #cv2.imshow("output", frame)#frame
    i=i+1
    print(i)
    # saving results and used parameters row wise during loop into a dataframe
    new_row = [i+1, roi[0],roi[1], roi[2], roi[3], circle1, circle2,circle3, videofilename, name,
               alpha, beta, dp, minDist, param1, param2, minRadius, maxRadius]
    df.loc[len(df)] = new_row

# cleaning up
out.release()
cap.release()
cv2.destroyAllWindows()

# filename and saving dataframe as cvs file       
filename =  './Output/timeseries/airsacradius_results_' + nameforfiles + '.csv'
df.to_csv(filename, sep = ',')


#smoothing of x,y and r of circles
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.filtfilt.html
#filename =  './Output/timeseries/airsacradius_results_' + nameforfiles + '.csv'
dfraw = pd.read_csv(filename)

# Opens the Video file
cap = cv2.VideoCapture(videofolder+videofilename)
fps = cap.get(cv2.CAP_PROP_FPS) 
frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

#outputfile smoothed
out_smoothed = cv2.VideoWriter('./Output/videos/output_smoothed'+videofilename,cv2.VideoWriter_fourcc(*'MP4V'), fps, 
                      (int(frameWidth), int(frameHeight)))

#delete outliers
#how? dynamic? mean +- 2*std of position of centroid in x or y direction? everything under or over that will be deleted
# calculate euclidean distance? to take x and y into account? 

# filter version using buterworth filter (low pass 10 Hz)
low = 12/(fps/2)
b, a = scipy.signal.butter(15, low,'low')
        
smooth_x = scipy.signal.filtfilt(b, a, dfraw['x'], axis=- 1, padtype='odd', padlen=None, method='pad', irlen=None)  
smooth_y = scipy.signal.filtfilt(b, a, dfraw['y'], axis=- 1, padtype='odd', padlen=None, method='pad', irlen=None)
smooth_r = scipy.signal.filtfilt(b, a, dfraw['r'], axis=- 1, padtype='odd', padlen=None, method='pad', irlen=None)

#filter version using Savitzky-golay filter
#wl = 11 #windowlength for savgol filter, must be odd
#p = 2   # polynomial for savgol filter, must be less than window_length

roi = [dfraw['roi_x1'][0], dfraw['roi_y1'][1], dfraw['roi_x2'][2],dfraw['roi_y2'][3]]

raw_x = dfraw['x']
raw_y = dfraw['y']
raw_r = dfraw['r']

#smooth_x = signal.savgol_filter(dfraw['x'], window_length=wl, polyorder=p)
#smooth_y = signal.savgol_filter(dfraw['y'], window_length=wl, polyorder=p)
#smooth_r = signal.savgol_filter(dfraw['r'], window_length=wl, polyorder=p)

name = dfraw['input_id']
circles_save = list(np.column_stack((smooth_x, smooth_y, smooth_r, name)))
raw_circles = list(np.column_stack((dfraw['x'], dfraw['y'], dfraw['r'])))

column_names = ['x', 'y', 'r', 'inputfile']


df2 = pd.DataFrame(circles_save, columns = column_names)
filename =  './Output/timeseries/airsacradius_results_smoothed' + videofilename + '.csv'
df2.to_csv(filename, sep = ',')

i = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    
    if ret == False:
        break
    output=frame.copy()
    x =int(smooth_x[i])
    y =int(smooth_y[i])
    r =int(smooth_r[i])
    #a,b,c = raw_circles[i]
    rx =int(raw_x[i])
    ry =int(raw_y[i])
    rr =int(raw_r[i])
    #roi
    cv2.rectangle(output,(int(roi[0]), int(roi[1])), (int(roi[0]+roi[2]), int(roi[1]+roi[3])), (0,0,0), 3)
    #unsmoothedcircle
    cv2.circle(output, (rx, ry), rr, (200, 0, 0), 1)
    #smoothed cicle
    cv2.circle(output, (x, y), r, (0,0, 255), 3)
    i= i+1
            
    #cv2.imshow("output", output)
    out_smoothed.write(output)

# cleaning up
out_smoothed.release()
cap.release()
cv2.destroyAllWindows()

#other filter option: savgoy filter
#from scipy.signal import savgol_filter
#def savgol(x, wl=14, p=2):
#    return savgol_filter(x, window_length=wl, polyorder=p)


# to do: filtering is too heavy with (8,0.125) check other combinations, also check whether filter is set correctly for
# our sampling rate --> changed that, fs is now in the function, what does the 8 mean? 
# automatic roi seems to be amiss quite heavily for all videos but 16_20...check again how to improve
# we need to read in roi from the data sheet, otherwise weird things can happen

parameters set
preprocessing image
done
1
preprocessing image
done
2
preprocessing image
done
3
preprocessing image
done
4
preprocessing image
done
5
preprocessing image
done
6
preprocessing image
done
7
preprocessing image
done
8
preprocessing image
done
9
preprocessing image
done
10
preprocessing image
done
11
preprocessing image
done
12
preprocessing image
done
13
preprocessing image
done
14
preprocessing image
done
15
preprocessing image
done
16
preprocessing image
done
17
preprocessing image
done
18
preprocessing image
done
19
preprocessing image
done
20
preprocessing image
done
21
preprocessing image
done
22
preprocessing image
done
23
preprocessing image
done
24
preprocessing image
done
25
preprocessing image
done
26
preprocessing image
done
27
preprocessing image
done
28
preprocessing image
done
29
preprocessing image
done
30
preprocessing image
done
31
preprocessing image
done
32
preprocessing image
done
33
preprocessing image
done
34
preprocessing image
done
35
preprocessing 

done
287
preprocessing image
done
288
preprocessing image
done
289
preprocessing image
done
290
preprocessing image
done
291
preprocessing image
done
292
preprocessing image
done
293
preprocessing image
done
294
preprocessing image
done
295
preprocessing image
done
296
preprocessing image
done
297
preprocessing image
done
298
preprocessing image
done
299
preprocessing image
done
300
preprocessing image
done
301
preprocessing image
done
302
preprocessing image
done
303
preprocessing image
done
304
preprocessing image
done
305
preprocessing image
done
306
preprocessing image
done
307
preprocessing image
done
308
preprocessing image
done
309
preprocessing image
done
310
preprocessing image
done
311
preprocessing image
done
312
preprocessing image
done
313
preprocessing image
done
314
preprocessing image
done
315
preprocessing image
done
316
preprocessing image
done
317
preprocessing image
done
318
preprocessing image
done
319
preprocessing image
done
320
preprocessing image
done
321
prepr

preprocessing image
done
571
preprocessing image
done
572
preprocessing image
done
573
preprocessing image
done
574
preprocessing image
done
575
preprocessing image
done
576
preprocessing image
done
577
preprocessing image
done
578
preprocessing image
done
579
preprocessing image
done
580
preprocessing image
done
581
preprocessing image
done
582
preprocessing image
done
583
preprocessing image
done
584
preprocessing image
done
585
preprocessing image
done
586
preprocessing image
done
587
preprocessing image
done
588
preprocessing image
done
589
preprocessing image
done
590
preprocessing image
done
591
preprocessing image
done
592
preprocessing image
done
593
preprocessing image
done
594
preprocessing image
done
595
preprocessing image
done
596
preprocessing image
done
597
preprocessing image
done
598
preprocessing image
done
599
preprocessing image
done
600
preprocessing image
done
601
preprocessing image
done
602
preprocessing image
done
603
preprocessing image
done
604
preprocessing 

done
854
preprocessing image
done
855
preprocessing image
done
856
preprocessing image
done
857
preprocessing image
done
858
preprocessing image
done
859
preprocessing image
done
860
preprocessing image
done
861
preprocessing image
done
862
preprocessing image
done
863
preprocessing image
done
864
preprocessing image
done
865
preprocessing image
done
866
preprocessing image
done
867
preprocessing image
done
868
preprocessing image
done
869
preprocessing image
done
870
preprocessing image
done
871
preprocessing image
done
872
preprocessing image
done
873
preprocessing image
done
874
preprocessing image
done
875
preprocessing image
done
876
preprocessing image
done
877
preprocessing image
done
878
preprocessing image
done
879
preprocessing image
done
880
preprocessing image
done
881
preprocessing image
done
882
preprocessing image
done
883
preprocessing image
done
884
preprocessing image
done
885
preprocessing image
done
886
preprocessing image
done
887
preprocessing image
done
888
prepr

1132
preprocessing image
done
1133
preprocessing image
done
1134
preprocessing image
done
1135
preprocessing image
done
1136
preprocessing image
done
1137
preprocessing image
done
1138
preprocessing image
done
1139
preprocessing image
done
1140
preprocessing image
done
1141
preprocessing image
done
1142
preprocessing image
done
1143
preprocessing image
done
1144
preprocessing image
done
1145
preprocessing image
done
1146
preprocessing image
done
1147
preprocessing image
done
1148
preprocessing image
done
1149
preprocessing image
done
1150
preprocessing image
done
1151
preprocessing image
done
1152
preprocessing image
done
1153
preprocessing image
done
1154
preprocessing image
done
1155
preprocessing image
done
1156
preprocessing image
done
1157
preprocessing image
done
1158
preprocessing image
done
1159
preprocessing image
done
1160
preprocessing image
done
1161
preprocessing image
done
1162
preprocessing image
done
1163
preprocessing image
done
1164
preprocessing image
done
1165
prepr

# Loop through all videos in the video folder

0.8

2.4